# Analysis with the Python API for ArcGIS

## What you should know from before:
    
    - What a GIS object is
    - How to add, remove and update content
    - How to publish content
    - Other basic function

## Login to Portal

- Recall that the JHU portal uses enterprise authentication.
- Leverage Pro's login ability to access the AGOL site

In [6]:
from arcgis.gis import GIS
gis = GIS("pro")
gis.users.me.username

'achapko1_GISandData'

## Feature Analysis

- A collection of Publish Geoprocessing tool hosted on AGOL or a Portal
- Tools cover a wide range of operations powered by a `WebGIS`
- Tools can output a Feature Layer or Feature Collection (in-memory dataset)

In [7]:
from arcgis.features import (analysis, analyze_patterns, 
                             enrich_data, elevation, 
                             use_proximity, hydrology)

## AGOL and Credits

- Digital spatial currency
- Provide premium content:
    + geocoding
    + analysis
    + geoenrichment

## How Much Does Analysis Cost?

- Each `Feature Analysis` tool has the ability to **estimate** the number of credits for any task
- It is good practice to see how much a tools **costs** before it is run

In [8]:
item = gis.content.get("c487cbc71d9c4372b7b76bd84ce0da68")
fl = item.layers[0]
credits = analyze_patterns.calculate_density(input_layer=fl,
                                             estimate=True)
f"This task costs this many credits: {credits}"

'This task costs this many credits: 0.967'

## To Wait or to not....

<img src="https://dynaimage.cdn.cnn.com/cnn/q_auto,w_380,c_fit/http%3A%2F%2Fcdn.cnn.com%2Fcnnnext%2Fdam%2Fassets%2F181213143815-gettyimages-884145174.jpg" />

- Analysis, Geoprocessing, and Raster Tools within the API can be run asynchronously
- Set `future=True` to get back a `Job` object to obtain the results at a later date

### When Not to Wait? (in general)

1. Performing multiple tasks
2. The next task isn't dependent on the previous task


## Using Future=True GP Tool Example

In [9]:
import time
from arcgis.geoprocessing import import_toolbox
tbx = import_toolbox("https://sampleserver6.arcgisonline.com/arcgis/rest/services/911CallsHotspot/GPServer")

In [10]:
fut = tbx.execute_911_calls_hotspot(future=True)
while fut.done() is not True:
    print('waiting')
    time.sleep(3)
result = fut.result()
lyr = result.result_layer

waiting
waiting
waiting
waiting


## Performing Multiple Tasks at Once

Let's view an example of building an asynchronous workflow with the Python API for ArcGIS

In [23]:
item = gis.content.search('title:CrimeAnalysisData owner:lscott_ANGP', 'feature layer', outside_org=True)[0]
lyrs = item.layers
liquor = lyrs[1] # liquor stores
phs = lyrs[0] # schools
crime = lyrs[2]

In [26]:
crime_sdf = crime.query(as_df=True)
crime_sdf.Description.value_counts()

STRONGARM - NO WEAPON                                          3617
AGGRAVATED: HANDGUN                                            3447
ARMED: HANDGUN                                                 3413
AGGRAVATED: OTHER DANG WEAPON                                  2864
AGGRAVATED:KNIFE/CUTTING INSTR                                 2094
AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON                  860
NON-AGGRAVATED                                                  714
AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST                 669
AGGRAVATED                                                      575
ARMED: OTHER DANGEROUS WEAPON                                   479
FIRST DEGREE MURDER                                             415
ATTEMPT: STRONGARM-NO WEAPON                                    368
ARMED:KNIFE/CUTTING INSTRUMENT                                  353
ATTEMPT: ARMED-HANDGUN                                          285
AGGRAVATED VEHICULAR HIJACKING                  

In [22]:
m = phs.query(as_df=True).spatial.plot()
m.add_layer(liquor)
m

MapView(layout=Layout(height='400px', width='100%'))

## Displaying the Results

In [48]:
m = gis.map()
m.extent = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
 'xmin': -13681103.700669535,'ymin': 5693908.708222669,
 'xmax': -13662854.360166471,'ymax': 5701552.4110511765}
m.add_layer(result.result_layer)
m

MapView(layout=Layout(height='400px', width='100%'))

## Data is like coal....

<img src="http://atlas-content-cdn.pixelsquid.com/stock-images/lump-of-coal-oJBoODC-600.jpg" width=30%/>

- Apply enough pressure and have the right conditions you get diamonds (or insights)

## Tidy Data Concept

<img src="https://r4ds.had.co.nz/images/tidy-1.png"/>

There are three rules which make a dataset tidy:

1. Each variable must have its own column.

2. Each observation must have its own row.

3. Each value must have its own cell.


## Pulling Data into DataFrames

